# Asymmetric Bounded-Confidence Model for Opinion Dynamics

This notebook demonstrates the Asymmetric Bounded-Confidence Agent-Based Model for studying opinion bubble stability.

**Key Hypothesis**: Higher perception asymmetry (alpha) leads to more stable opinion bubbles because agents discount outgroup signals, preventing cluster merging.

## Method Overview

The model modulates confidence bounds based on perception asymmetry:
- **Baseline (α=0)**: Standard bounded-confidence model - agents interact equally with all neighbors within their confidence bound
- **Method (α>0)**: Asymmetric model - agents perceive their ingroup as more diverse than outgroups, discounting outgroup signals by factor (1-α)

## Setup and Data Loading

In [ ]:
# Install dependencies if needed
try:
    import pandas as pd
    import matplotlib.pyplot as plt
except ImportError:
    !pip install pandas matplotlib
    import pandas as pd
    import matplotlib.pyplot as plt

import json
from urllib.request import urlopen
from urllib.error import URLError
import os

In [ ]:
# Data loading with GitHub URL and local fallback
GITHUB_DATA_URL = "https://raw.githubusercontent.com/AMGrobelnik/ai-invention-ab58ab-perception-asymmetry-feedback-loop-how-d/main/asymmetric_bounded_confidence_model_for_opinion_dynamics_wit/demo/demo_data.json"
LOCAL_DATA_PATH = "demo_data.json"

def load_data():
    """Load data from GitHub URL with local fallback."""
    # Try GitHub URL first
    try:
        print(f"Attempting to load data from GitHub...")
        with urlopen(GITHUB_DATA_URL, timeout=10) as response:
            data = json.loads(response.read().decode('utf-8'))
            print(f"✓ Successfully loaded data from GitHub")
            return data
    except (URLError, Exception) as e:
        print(f"Could not load from GitHub: {e}")
    
    # Fall back to local file
    if os.path.exists(LOCAL_DATA_PATH):
        print(f"Loading from local file: {LOCAL_DATA_PATH}")
        with open(LOCAL_DATA_PATH, 'r') as f:
            data = json.load(f)
            print(f"✓ Successfully loaded data from local file")
            return data
    
    raise FileNotFoundError("Could not load data from GitHub or local file")

# Load the data
data = load_data()
examples = data['examples']
print(f"\nLoaded {len(examples)} experiment examples")

## Parse and Prepare Data

In [ ]:
# Parse the prediction results from JSON strings
records = []
for ex in examples:
    baseline = json.loads(ex['predict_baseline'])
    method = json.loads(ex['predict_method'])
    
    records.append({
        'seed': ex['context']['seed'],
        'method_alpha': ex['context']['method_alpha'],
        'baseline_lifetime': baseline['cluster_lifetime_mean'],
        'method_lifetime': method['cluster_lifetime_mean'],
        'baseline_clusters': baseline['final_cluster_count'],
        'method_clusters': method['final_cluster_count'],
        'baseline_variance': baseline['final_variance'],
        'method_variance': method['final_variance'],
        'baseline_convergence': baseline['convergence_time'],
        'method_convergence': method['convergence_time'],
        'correlation_r': ex['context']['correlation_r'],
        't_pvalue': ex['context']['t_pvalue']
    })

df = pd.DataFrame(records)
print("Parsed experiment results:")
df.head(10)

## Visualize: Cluster Lifetime vs Alpha

In [ ]:
# Aggregate by alpha value
alpha_grouped = df.groupby('method_alpha').agg({
    'method_lifetime': 'mean',
    'baseline_lifetime': 'mean',
    'method_clusters': 'mean',
    'baseline_clusters': 'mean'
}).reset_index()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Cluster Lifetime vs Alpha
ax1 = axes[0]
ax1.scatter(df['method_alpha'], df['method_lifetime'], alpha=0.6, label='Method (α>0)', color='blue')
ax1.axhline(y=df['baseline_lifetime'].mean(), color='red', linestyle='--', linewidth=2, label=f'Baseline (α=0) avg: {df["baseline_lifetime"].mean():.1f}')
ax1.plot(alpha_grouped['method_alpha'], alpha_grouped['method_lifetime'], 'b-', linewidth=2, alpha=0.7)
ax1.set_xlabel('Perception Asymmetry (α)', fontsize=12)
ax1.set_ylabel('Cluster Lifetime (timesteps)', fontsize=12)
ax1.set_title('Opinion Bubble Stability vs Perception Asymmetry', fontsize=13)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Final Cluster Count vs Alpha
ax2 = axes[1]
ax2.scatter(df['method_alpha'], df['method_clusters'], alpha=0.6, label='Method (α>0)', color='green')
ax2.axhline(y=df['baseline_clusters'].mean(), color='red', linestyle='--', linewidth=2, label=f'Baseline (α=0) avg: {df["baseline_clusters"].mean():.1f}')
ax2.plot(alpha_grouped['method_alpha'], alpha_grouped['method_clusters'], 'g-', linewidth=2, alpha=0.7)
ax2.set_xlabel('Perception Asymmetry (α)', fontsize=12)
ax2.set_ylabel('Final Cluster Count', fontsize=12)
ax2.set_title('Opinion Fragmentation vs Perception Asymmetry', fontsize=13)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Statistical Analysis

In [ ]:
# Display key statistics
correlation_r = df['correlation_r'].iloc[0]
t_pvalue = df['t_pvalue'].iloc[0]

print("=" * 60)
print("STATISTICAL RESULTS")
print("=" * 60)
print(f"\nCorrelation (α vs Lifetime): r = {correlation_r:.4f}")
print(f"T-test p-value (Baseline vs Method): p = {t_pvalue:.6f}")
print(f"\nBaseline (α=0) Mean Lifetime: {df['baseline_lifetime'].mean():.2f} timesteps")
print(f"Method (α>0) Mean Lifetime: {df['method_lifetime'].mean():.2f} timesteps")
print(f"\nLifetime Increase: {((df['method_lifetime'].mean() / df['baseline_lifetime'].mean()) - 1) * 100:.1f}%")

print("\n" + "=" * 60)
print("INTERPRETATION")
print("=" * 60)
if correlation_r > 0 and t_pvalue < 0.05:
    print("\n✓ Hypothesis SUPPORTED: Higher perception asymmetry (α)")
    print("  leads to more stable opinion bubbles (longer cluster lifetimes).")
    print(f"\n  The positive correlation (r={correlation_r:.3f}) and significant")
    print(f"  t-test (p={t_pvalue:.4f}) indicate that agents who discount")
    print("  outgroup signals maintain more persistent opinion clusters.")
else:
    print("\n✗ Results inconclusive or hypothesis not supported.")

## Baseline vs Method Comparison

In [ ]:
# Compare method vs baseline across different alpha values
fig, ax = plt.subplots(figsize=(10, 6))

# Calculate lifetime improvement for each example
df['lifetime_improvement'] = df['method_lifetime'] - df['baseline_lifetime']

colors = ['green' if x > 0 else 'red' for x in df['lifetime_improvement']]
ax.bar(range(len(df)), df['lifetime_improvement'], color=colors, alpha=0.7)
ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax.set_xlabel('Experiment Index', fontsize=12)
ax.set_ylabel('Lifetime Improvement (Method - Baseline)', fontsize=12)
ax.set_title('Cluster Lifetime Improvement: Method vs Baseline', fontsize=13)
ax.grid(True, alpha=0.3, axis='y')

# Add annotation
positive_count = (df['lifetime_improvement'] > 0).sum()
total_count = len(df)
ax.annotate(f'{positive_count}/{total_count} experiments show improvement', 
            xy=(0.02, 0.98), xycoords='axes fraction', fontsize=11,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

## Summary Statistics Table

In [ ]:
# Summary table by alpha ranges
df['alpha_range'] = pd.cut(df['method_alpha'], bins=[0, 0.5, 1.0, 1.5], 
                           labels=['Low (0-0.5)', 'Medium (0.5-1.0)', 'High (1.0-1.5)'])

summary = df.groupby('alpha_range', observed=True).agg({
    'method_lifetime': ['mean', 'std'],
    'baseline_lifetime': ['mean', 'std'],
    'lifetime_improvement': ['mean', 'std']
}).round(2)

print("\nSummary by Alpha Range:")
print(summary)

## Conclusion

This demonstration shows that the Asymmetric Bounded-Confidence Model produces measurably more stable opinion clusters compared to the standard baseline model. The key mechanism is:

1. **Ingroup Preference**: Agents with α > 0 discount information from outgroup members
2. **Cluster Stability**: This leads to longer-lasting opinion bubbles
3. **Statistical Significance**: The difference between baseline and method is statistically significant

These findings have implications for understanding how perception biases in social networks can contribute to opinion polarization and echo chamber formation.